# Bain Strategic Challenge

In [28]:
import pandas as pd
import datetime

In [60]:
'''
Unfortunately, the way the destinated_area columns is made makes it impossible to simply use
pd.read_csv('fn')
as it is not able to interpret lines like
- number,number
- number,
- ,
simultaneously.

This can be remedied by manually extracting the data from the file and treating some edge cases.
We also take the chance to convert the dates to a Timestamp, which allows for some better ease of use later.
'''

# Important to use utf-8, as 'açaí' will need a better encoding than default ascii
with open('historical-database.csv', 'r', encoding='utf-8') as f:
    data = f.read()

data = data.split('\n')
# Separating columns manually
columns = data[0]
data = data[1:-1]
# Remove ending comma, then split by semi-colon
columns = columns[:-1].split(';')

def process_line(line: str) -> list:
    entries = line.split(';')
    if ',' in entries[-1] and len(entries[-1])>1:
        entries[-1] = float(entries[-1].replace(',','.',-1))
    elif len(entries[-1]) <= 1:
        entries[-1] = None
    else:
        entries[-1] = float(entries[-1])
    # Convert to python-friendly date format.
    entries[0] = datetime.datetime.strptime(entries[0],"%d/%m/%Y")
    return entries
data = list(map(process_line, data))
df = pd.DataFrame(data=data, columns=columns)
df


,year,city_code,product_type,product,destinated_area
0,1974-01-01,960b4f2c94a2fb2c,temporary,Others,0.0000
1,1974-01-01,746cc42bfb8f6b62,temporary,Others,0.0000
2,1974-01-01,6cce2bf873870afc,temporary,Others,0.0000
3,1974-01-01,4de42e351006a2ae,temporary,Others,0.0000
4,1974-01-01,5b6072f8f6d37acc,temporary,Others,0.0000
...,...,...,...,...,...
73739,2013-01-01,74d75dde6dc4a5ec,pasture,Livestock,150866.5644
73740,2014-01-01,74d75dde6dc4a5ec,pasture,Livestock,152446.3960
73741,2015-01-01,74d75dde6dc4a5ec,pasture,Livestock,154681.0205
73742,2016-01-01,74d75dde6dc4a5ec,pasture,Livestock,164706.2778


In [65]:
# Useful to know what kinds of values can be found in each column
possible_vals = {col:df[col].value_counts().index.to_list() for col in ['product_type', 'product']}
possible_vals

{'product_type': ['temporary', 'permanent', 'pasture'],
 'product': ['Others',
  'Rice',
  'Beans',
  'Cassava',
  'Corn',
  'Cocoa',
  'Palm oil',
  'Açaí',
  'Sorghum',
  'Soy',
  'Livestock']}

In [75]:
# Separate by product type
df_temporary = df[df['product_type'] == 'temporary'].reset_index().drop('index',axis=1)
df_permanent = df[df['product_type'] == 'permanent'].reset_index().drop('index',axis=1)
df_pasture = df[df['product_type'] == 'pasture'].reset_index().drop('index',axis=1)
df_temporary

,year,city_code,product_type,product,destinated_area
0,1974-01-01,960b4f2c94a2fb2c,temporary,Others,0.0
1,1974-01-01,746cc42bfb8f6b62,temporary,Others,0.0
2,1974-01-01,6cce2bf873870afc,temporary,Others,0.0
3,1974-01-01,4de42e351006a2ae,temporary,Others,0.0
4,1974-01-01,5b6072f8f6d37acc,temporary,Others,0.0
...,...,...,...,...,...
43819,2017-01-01,64a227aa24370b17,temporary,Beans,7.0
43820,2017-01-01,64a227aa24370b17,temporary,Cassava,222.0
43821,2017-01-01,64a227aa24370b17,temporary,Corn,2100.0
43822,2017-01-01,64a227aa24370b17,temporary,Soy,NaN


In [76]:
df_permanent

,year,city_code,product_type,product,destinated_area
0,1974-01-01,960b4f2c94a2fb2c,permanent,Others,0.0
1,1974-01-01,746cc42bfb8f6b62,permanent,Others,0.0
2,1974-01-01,6cce2bf873870afc,permanent,Others,0.0
3,1974-01-01,4de42e351006a2ae,permanent,Others,0.0
4,1974-01-01,5b6072f8f6d37acc,permanent,Others,0.0
...,...,...,...,...,...
25163,2001-01-01,164c8953d3af2bbe,permanent,Cocoa,825.0
25164,2001-01-01,f1fd357c5adaa8dc,permanent,Palm oil,NaN
25165,2001-01-01,8a4ab531676691da,permanent,Palm oil,NaN
25166,2001-01-01,c45d8ddba643c0d7,permanent,Açaí,NaN


In [77]:
df_pasture

,year,city_code,product_type,product,destinated_area
0,1985-01-01,960b4f2c94a2fb2c,pasture,Livestock,20960.60904
1,1986-01-01,960b4f2c94a2fb2c,pasture,Livestock,10813.07188
2,1987-01-01,960b4f2c94a2fb2c,pasture,Livestock,15996.71607
3,1988-01-01,960b4f2c94a2fb2c,pasture,Livestock,19146.77781
4,1989-01-01,960b4f2c94a2fb2c,pasture,Livestock,18236.41930
...,...,...,...,...,...
4747,2013-01-01,74d75dde6dc4a5ec,pasture,Livestock,150866.56440
4748,2014-01-01,74d75dde6dc4a5ec,pasture,Livestock,152446.39600
4749,2015-01-01,74d75dde6dc4a5ec,pasture,Livestock,154681.02050
4750,2016-01-01,74d75dde6dc4a5ec,pasture,Livestock,164706.27780


In [79]:
# Save subdivided dataframes to different files
df_pasture.to_csv('subdivisions/pasture_database.csv')
df_temporary.to_csv('subdivisions/temporary_database.csv')
df_permanent.to_csv('subdivisions/permanent_database.csv')